## Uso General de CPLEX:

In [20]:
from pulp import LpProblem, LpVariable, LpMinimize, CPLEX_CMD, lpSum

# =============================
# 1. CREACIÓN DEL MODELO
# =============================
# Se crea un modelo de optimización lineal con objetivo de minimización.
# Alternativa: cambiar a LpMaximize si se busca maximizar.
model = LpProblem("Ejemplo_WTA", LpMinimize)

# =============================
# 2. DEFINICIÓN DE VARIABLES
# =============================
# Se crean variables continuas x e y.
# Parámetros ajustables:
# - lowBound: cota inferior de la variable (puede ser negativa si se desea)
# - upBound: se puede definir un límite superior si aplica
# - cat: tipo de variable
#     "Continuous" → variable continua
#     "Integer"    → variable entera
#     "Binary"     → variable binaria (0 o 1)
x = LpVariable("x", lowBound=0, cat="Continuous")
y = LpVariable("y", lowBound=0, cat="Continuous")

# =============================
# 3. DEFINICIÓN DE LA FUNCIÓN OBJETIVO
# =============================
# La función objetivo busca minimizar el costo total Z = 2x + 3y.
# Este bloque se puede modificar fácilmente para representar costos, beneficios, distancias, etc.
model += 2 * x + 3 * y, "Costo_Total"

# =============================
# 4. RESTRICCIONES
# =============================
# Se agregan dos restricciones lineales al modelo.
# El lado derecho (10 y 12) y los coeficientes son personalizables.
# Puedes usar >=, <=, == según el problema.
model += x + 2 * y >= 10, "Restriccion1"
model += 3 * x + y >= 12, "Restriccion2"

# =============================
# 5. CONFIGURACIÓN DEL SOLVER CPLEX
# =============================

# Ruta explícita al binario de CPLEX:
cplex_path = "/Applications/CPLEX_Studio_Community2211/cplex/bin/arm64_osx/cplex"

# =============================
# OPCIONES DE CPLEX
# =============================
# Estas líneas controlan el comportamiento interno del solver:

solver = CPLEX_CMD(
    path=cplex_path,  # Ruta al ejecutable de CPLEX
    msg=True,         # Mostrar el log completo del solver (útil para depuración)
    timeLimit=60,     # Límite de tiempo en segundos para resolver el problema
    options=[
        # ---- TOLERANCIAS ----
        # Define cuán cerca de la solución óptima se permite aceptar.
        "mip tolerances mipgap 0.01",       # Gap relativo: 1% → (Z - Z*) / Z* <= 0.01
        "mip tolerances absmipgap 1e-4",    # Gap absoluto permitido en la función objetivo
        "mip tolerances integrality 1e-5",  # Precisión para que una variable se considere entera

        # ---- HEURÍSTICAS ----
        # CPLEX aplica métodos aproximados para encontrar soluciones viables rápido.
        "mip strategy heuristicfreq 5",     # Ejecuta heurísticas cada 5 nodos de B&B
        "mip strategy lbheur 1",            # Activar heurística basada en el bound inferior

        # ---- BRANCHING STRATEGIES ----
        # Controla cómo se seleccionan las variables al ramificar.
        "mip strategy variableselect 3",    # Strong branching (más lento pero más robusto)
        "mip strategy branch 1",            # Depth-first branching

        # ---- CORTES (CUTS) ----
        # Permiten acotar el espacio de búsqueda añadiendo restricciones implícitas
        "mip cuts gomory 2",                # Cortes de Gomory agresivos
        "mip cuts mircut 2",                # Mixed Integer Rounding Cuts
        "mip cuts cover 2",                 # Cover cuts
        "mip cuts clique 2",                # Clique cuts
        "mip cuts zerohalf 2",              # Zero-half cuts

        # ---- PREPROCESSING ----
        "preprocessing presolve 1",         # Activa el presolve general
        "mip strategy presolvenode 1",      # Aplica presolve también en cada nodo del árbol

        # ---- PARALELISMO ----
        "parallel 1",                       # Modo determinista (misma solución en cada corrida)

        # ---- LÍMITES ADICIONALES ----
        "mip limits nodes 1000",            # Máximo de 1000 nodos en el árbol de búsqueda
        "mip limits solutions 100"          # Máximo de 100 soluciones factibles almacenadas
    ]
)

# =============================
# 6. RESOLUCIÓN DEL MODELO
# =============================
status = model.solve(solver)

# =============================
# 7. RESULTADOS
# =============================
# Estado: 1 = óptimo, 0 = no resuelto, -1 = error, etc.
print("Estado:", model.status)

# Mostrar los valores de las variables
print("x =", x.varValue)
print("y =", y.varValue)

# Valor óptimo de la función objetivo
print("Z =", model.objective.value())



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer Community Edition 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Problem '/var/folders/sh/2jnszg1d0llfrs9_1_8prq9r0000gn/T/afc1991f4a7b4b4a88cf438a0cfa6cad-pulp.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> New value for time limit in seconds: 60
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.
CPLEX> Not a mixed integer problem.

## Uso General de CBC_CMD

In [26]:
from pulp import LpProblem, LpVariable, LpMinimize, PULP_CBC_CMD

# =============================
# 1. CREACIÓN DEL MODELO
# =============================
# Se crea un modelo de optimización lineal.
# El objetivo es de tipo minimización (puedes cambiar a LpMaximize si deseas maximizar).
model = LpProblem("Ejemplo_WTA_CBC", LpMinimize)

# =============================
# 2. DEFINICIÓN DE VARIABLES
# =============================
# Se crean dos variables continuas: x e y.
# Argumentos importantes:
# - lowBound: cota inferior (por defecto es None → variable no acotada inferiormente).
# - upBound: también puedes definir una cota superior (opcional).
# - cat: tipo de variable ("Continuous", "Integer", "Binary").
x = LpVariable("x", lowBound=0, cat="Continuous")
y = LpVariable("y", lowBound=0, cat="Continuous")

# =============================
# 3. DEFINICIÓN DE LA FUNCIÓN OBJETIVO
# =============================
# La función objetivo es lineal: minimizar Z = 2x + 3y.
model += 2 * x + 3 * y, "Costo_Total"

# =============================
# 4. RESTRICCIONES
# =============================
# Se definen dos restricciones lineales.
# Puedes usar >=, <=, == según el tipo de condición que necesites.
model += x + 2 * y >= 10, "Restriccion1"
model += 3 * x + y >= 12, "Restriccion2"

# =============================
# 5. CONFIGURACIÓN DEL SOLVER CBC (open-source)
# =============================
# CBC es un solver gratuito y multiplataforma, integrado con PuLP.
# Es útil para problemas de tamaño pequeño o mediano (LP y MIP).
# Puede usarse directamente sin necesidad de una instalación externa si se usa PULP_CBC_CMD.

# Parámetros disponibles en PULP_CBC_CMD:
# - msg: True/False → Muestra la salida detallada del solver (útil para depuración).
# - timeLimit: Límite de tiempo en segundos para abortar la búsqueda (default: sin límite).
# - gapRel: Gap relativo (0.01 = permite soluciones hasta 1% peores que el óptimo).
# - gapAbs: Gap absoluto (en unidades del valor de la función objetivo).
# - threads: Número de hilos (CPUs) a utilizar → threads=1 asegura reproducibilidad.

solver = PULP_CBC_CMD(
    msg=True,             # Mostrar salida del solver (recomendado en fase de desarrollo)
    timeLimit=60,         # Límite de tiempo: 60 segundos
    gapRel=0.01,          # Tolerancia relativa de optimalidad (1%)
    gapAbs=1e-4,          # Tolerancia absoluta sobre el valor óptimo
    threads=1             # Usar un solo hilo (determinismo, útil para resultados reproducibles)
)

# =============================
# 6. RESOLUCIÓN DEL MODELO
# =============================
# Se ejecuta el solver con la configuración indicada.
# El resultado se almacena en la variable `status` (puede ser usado para control condicional).
status = model.solve(solver)

# =============================
# 7. RESULTADOS
# =============================
# Imprimir estado de la solución:
# - 1: Óptimo encontrado
# - 0: No solucionado
# - -1: Inviable
# - -2: Sin solución factible
print("Estado:", model.status)

# Mostrar los valores óptimos de las variables
print("x =", x.varValue)
print("y =", y.varValue)

# Mostrar el valor óptimo de la función objetivo
print("Z =", model.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/educarrascovidal/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sh/2jnszg1d0llfrs9_1_8prq9r0000gn/T/5fd78465f6ff4a629a552a4d36fca693-pulp.mps -sec 60 -ratio 0.01 -allow 0.0001 -threads 1 -timeMode elapsed -branch -printingOptions all -solution /var/folders/sh/2jnszg1d0llfrs9_1_8prq9r0000gn/T/5fd78465f6ff4a629a552a4d36fca693-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 14 RHS
At line 17 BOUNDS
At line 18 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
ratioGap was changed from 0 to 0.01
allowableGap was changed from 1e-10 to 0.0001
threads was changed from 0 to 1
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
0  Obj 0 Primal inf 8.9999998 (2)
2  Obj 16.4
Optimal - ob